# <center>JSONiq Update Facility sandbox</center>


The following cell is meant for Google Colab, which does not have the right Java version.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk
!java -version

In [60]:
!pip install jsoniq


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


We now initialize a RumbleDB session and test that it works.

In [91]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.withDelta().getOrCreate();

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

({'foo': [42]},)


For convenience, we also activate the JSONiq magic.

In [92]:
%load_ext jsoniqmagic
# We will display the first 5 items. This parameter can be changed.
rumble.getRumbleConf().setResultSizeCap(20)

The jsoniqmagic extension is already loaded. To reload it, use:
  %reload_ext jsoniqmagic


JavaObject id=o514

We create a new collection

In [167]:
%%jsoniq -u -t
create collection table("foo") with for $i in 100 to 300 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.4261181354522705 ms


In [168]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|100|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
|101|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
|102|[Alice, Bob, Char...|      2|       2.0|                0|        |            foo|
|103|[Alice, Bob, Char...|      3|       3.0|                0|        |            foo|
|104|[Alice, Bob, Char...|      4|       4.0|                0|        |            foo|
|105|[Alice, Bob, Char...|      5|       5.0|                0|        |            foo|
|106|[Alice, Bob, Char...|      6|       6.0|                0|        |            foo|
|107|[Alice, Bob, Char...|      7|       7.0|                0|        |            foo|
|108|[Alice, Bob, Cha

We insert new data -- remember the snapshot semantics, all updates are computed against the original table, and then applied jointly.

In [169]:
%%jsoniq -u -t
insert for $i in 400 to 600 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection table("foo"),
insert for $i in 700 to 1000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection table("foo"),
insert for $i in 1100 to 1300 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before table("foo")[position() eq 2] into collection,
insert for $i in 1400 to 1600 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after table("foo")[position() eq 2] into collection


Updates applied successfully.
Response time: 3.0666849613189697 ms


In [170]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+-------------+-----------------+--------+---------------+
|foo|                 bar|__rowID|   __rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+-------------+-----------------+--------+---------------+
|400|[Alice, Bob, Char...|    603|-99504.950495|                0|        |            foo|
|401|[Alice, Bob, Char...|    604| -99009.90099|                0|        |            foo|
|402|[Alice, Bob, Char...|    605|-98514.851485|                0|        |            foo|
|403|[Alice, Bob, Char...|    606| -98019.80198|                0|        |            foo|
|404|[Alice, Bob, Char...|    607|-97524.752475|                0|        |            foo|
|405|[Alice, Bob, Char...|    608| -97029.70297|                0|        |            foo|
|406|[Alice, Bob, Char...|    609|-96534.653465|                0|        |            foo|
|407|[Alice, Bob, Char...|    610| -96039.60396|                0|        |     

We delete a record

In [171]:
%%jsoniq -u -t
delete table("foo")[$$.foo eq 1500] from collection

Updates applied successfully.
Response time: 0.6478841304779053 ms


In [172]:
%%jsoniq -df -t
count(table("foo"))

+-----+
|value|
+-----+
| 1104|
+-----+

Response time: 0.1739048957824707 ms


We edit a record

In [173]:
%%jsoniq -u -t
edit table("foo")[$$.foo eq 1400] into { "foo" : 1700, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 0.9577999114990234 ms


In [174]:
%%jsoniq
table("foo")[$$.foo eq 14002]

In [175]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+-------------+-----------------+--------+---------------+
|foo|                 bar|__rowID|   __rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+-------------+-----------------+--------+---------------+
|400|[Alice, Bob, Char...|    603|-99504.950495|                0|        |            foo|
|401|[Alice, Bob, Char...|    604| -99009.90099|                0|        |            foo|
|402|[Alice, Bob, Char...|    605|-98514.851485|                0|        |            foo|
|403|[Alice, Bob, Char...|    606| -98019.80198|                0|        |            foo|
|404|[Alice, Bob, Char...|    607|-97524.752475|                0|        |            foo|
|405|[Alice, Bob, Char...|    608| -97029.70297|                0|        |            foo|
|406|[Alice, Bob, Char...|    609|-96534.653465|                0|        |            foo|
|407|[Alice, Bob, Char...|    610| -96039.60396|                0|        |     

In [176]:
%%jsoniq -t
rename json table("foo")[1].foo as "bar2";

Response time: 1.0745837688446045 ms


In [177]:
%%jsoniq -df -t
table("foo")

+----+--------------------+-------+-------------+----+-----------------+--------+---------------+
| foo|                 bar|__rowID|   __rowOrder|bar2|__mutabilityLevel|__pathIn|__tableLocation|
+----+--------------------+-------+-------------+----+-----------------+--------+---------------+
|NULL|[Alice, Bob, Char...|    603|-99504.950495| 400|                0|        |            foo|
| 401|[Alice, Bob, Char...|    604| -99009.90099|NULL|                0|        |            foo|
| 402|[Alice, Bob, Char...|    605|-98514.851485|NULL|                0|        |            foo|
| 403|[Alice, Bob, Char...|    606| -98019.80198|NULL|                0|        |            foo|
| 404|[Alice, Bob, Char...|    607|-97524.752475|NULL|                0|        |            foo|
| 405|[Alice, Bob, Char...|    608| -97029.70297|NULL|                0|        |            foo|
| 406|[Alice, Bob, Char...|    609|-96534.653465|NULL|                0|        |            foo|
| 407|[Alice, Bob, C

In [178]:
%%jsoniq -u -t
delete collection table("foo")

Updates applied successfully.
Response time: 0.020318984985351562 ms


In [179]:
%%jsoniq -u -t
create collection delta-file("foobar") with for $i in 100 to 300 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.4199681282043457 ms


In [180]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|100|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
|101|[Alice, Bob, Char...|      1|       1.0|                0|        |delta.`file:/User...|
|102|[Alice, Bob, Char...|      2|       2.0|                0|        |delta.`file:/User...|
|103|[Alice, Bob, Char...|      3|       3.0|                0|        |delta.`file:/User...|
|104|[Alice, Bob, Char...|      4|       4.0|                0|        |delta.`file:/User...|
|105|[Alice, Bob, Char...|      5|       5.0|                0|        |delta.`file:/User...|
|106|[Alice, Bob, Char...|      6|       6.0|                0|        |delta.`file:/User...|
|107|[Alice, Bob, Char...|      7|       7.0|               

In [181]:
%%jsoniq -u -t
insert for $i in 400 to 600 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection delta-file("foobar"),
insert for $i in 700 to 1000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection delta-file("foobar"),
insert for $i in 1100 to 1300 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before delta-file("foobar")[position() eq 2] into collection,
insert for $i in 1400 to 1600 return { "foobar" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after delta-file("foobar")[position() eq 2] into collection


Updates applied successfully.
Response time: 2.686344861984253 ms


In [182]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|   __rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|400|[Alice, Bob, Char...|    603|-99504.950495|                0|        |delta.`file:/User...|
|401|[Alice, Bob, Char...|    604| -99009.90099|                0|        |delta.`file:/User...|
|402|[Alice, Bob, Char...|    605|-98514.851485|                0|        |delta.`file:/User...|
|403|[Alice, Bob, Char...|    606| -98019.80198|                0|        |delta.`file:/User...|
|404|[Alice, Bob, Char...|    607|-97524.752475|                0|        |delta.`file:/User...|
|405|[Alice, Bob, Char...|    608| -97029.70297|                0|        |delta.`file:/User...|
|406|[Alice, Bob, Char...|    609|-96534.653465|                0|        |delta.`file:/User...|
|407|[Alice, Bob, Char...|    

In [183]:
%%jsoniq -u -t
delete delta-file("foobar")[$$.foo eq 1500] from collection

Updates applied successfully.
Response time: 0.6650950908660889 ms


In [184]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|   __rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|400|[Alice, Bob, Char...|    603|-99504.950495|                0|        |delta.`file:/User...|
|401|[Alice, Bob, Char...|    604| -99009.90099|                0|        |delta.`file:/User...|
|402|[Alice, Bob, Char...|    605|-98514.851485|                0|        |delta.`file:/User...|
|403|[Alice, Bob, Char...|    606| -98019.80198|                0|        |delta.`file:/User...|
|404|[Alice, Bob, Char...|    607|-97524.752475|                0|        |delta.`file:/User...|
|405|[Alice, Bob, Char...|    608| -97029.70297|                0|        |delta.`file:/User...|
|406|[Alice, Bob, Char...|    609|-96534.653465|                0|        |delta.`file:/User...|
|407|[Alice, Bob, Char...|    

In [185]:
%%jsoniq -u -t
edit delta-file("foobar")[$$.foo eq 1400] into { "foo" : 1700, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 1.004621982574463 ms


In [186]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|   __rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+-------------+-----------------+--------+--------------------+
|400|[Alice, Bob, Char...|    603|-99504.950495|                0|        |delta.`file:/User...|
|401|[Alice, Bob, Char...|    604| -99009.90099|                0|        |delta.`file:/User...|
|402|[Alice, Bob, Char...|    605|-98514.851485|                0|        |delta.`file:/User...|
|403|[Alice, Bob, Char...|    606| -98019.80198|                0|        |delta.`file:/User...|
|404|[Alice, Bob, Char...|    607|-97524.752475|                0|        |delta.`file:/User...|
|405|[Alice, Bob, Char...|    608| -97029.70297|                0|        |delta.`file:/User...|
|406|[Alice, Bob, Char...|    609|-96534.653465|                0|        |delta.`file:/User...|
|407|[Alice, Bob, Char...|    

In [187]:
%%jsoniq -u -t
delete collection delta-file("foobar")

Updates applied successfully.
Response time: 0.022814035415649414 ms
